In [24]:
import mlflow
import mlflow.sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import datasets

In [25]:
mlflow.get_tracking_uri()

'http://localhost:5000'

In [26]:
mlflow.set_tracking_uri("http://localhost:5000")

In [27]:
mlflow.get_tracking_uri()

'http://localhost:5000'

In [28]:
mlflow.set_experiment("iris") # creates an experiment if it doesn't exist

<Experiment: artifact_location='mlflow-artifacts:/251967034232254676', creation_time=1705480194253, experiment_id='251967034232254676', last_update_time=1705480194253, lifecycle_stage='active', name='iris', tags={}>

In [22]:
iris = datasets.load_iris()
x = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

In [33]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [31]:
import joblib

with mlflow.start_run(run_name="Iris KNN Experiment") as run:
    
    # Add parameters for tuning
    num_neighbors = 4
    mlflow.log_param("num_neighbors", num_neighbors)

    # Train the model
    knn = KNeighborsClassifier(n_neighbors=num_neighbors)
    knn.fit(X_train, y_train)
    predictions = knn.predict(X_test)

    # Save the model artifact using joblib.dump
    model_path = "knn-model.joblib"
    joblib.dump(knn, model_path)
    mlflow.log_artifact(model_path, "model")

    # Log model performance 
    mse = mean_squared_error(y_test, predictions)
    mlflow.log_metric("mse", mse)
    print("  MSE: %f" % mse)

    run_id = run.info.run_uuid
    experiment_id = run.info.experiment_id
    
    # End the MLflow run
    mlflow.end_run()

    # Print artifact URI and run ID
    print(mlflow.get_artifact_uri())
    print("Run ID: %s" % run_id)

  MSE: 0.066667
mlflow-artifacts:/251967034232254676/bcd0ff02bc5d40dbaa4fde04e4f57b4f/artifacts
Run ID: dee1b645856b473e96f7460baaec3213


In [32]:
model_path = "knn-model.joblib"

# Chargemenet du modèle
loaded_model = joblib.load(model_path)

predictions = loaded_model.predict(X_test)
